In [1]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import mlflow
import mlflow.sklearn

# Cargar los datos procesados desde el notebook anterior
data = pd.read_csv('processed_data.csv')

# Mostrar las primeras filas del dataset
data.head()


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [2]:
# Seleccionar las características más importantes (según la ganancia de información mutua)
important_features = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_5', 'LIMIT_BAL']

# Separar las características predictoras (X) y la variable objetivo (y)
X = data[important_features]
y = data['default.payment.next.month']

# Dividir los datos en conjunto de entrenamiento (80%) y conjunto de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Configuración de MLFlow para rastrear el experimento
mlflow.set_experiment('Prediccion_Incumplimiento_Pagos')

with mlflow.start_run():
    # Crear y entrenar el modelo Random Forest
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Hacer predicciones en el conjunto de prueba
    y_pred = rf_model.predict(X_test)
    
    # Calcular las métricas de rendimiento
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Mostrar las métricas
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    # Mostrar reporte de clasificación
    print("Reporte de clasificación:")
    print(classification_report(y_test, y_pred))
    
    # Registrar las métricas en MLFlow
    mlflow.log_metric("Accuracy", accuracy)
    mlflow.log_metric("F1 Score", f1)
    
    # Registrar el modelo en MLFlow
    mlflow.sklearn.log_model(rf_model, "random_forest_model")
    
    # Finalizar el run de MLFlow
    mlflow.end_run()


2024/09/28 13:15:36 INFO mlflow.tracking.fluent: Experiment with name 'Prediccion_Incumplimiento_Pagos' does not exist. Creating a new experiment.


Accuracy: 0.8010
F1 Score: 0.4400
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.84      0.93      0.88      4687
           1       0.57      0.36      0.44      1313

    accuracy                           0.80      6000
   macro avg       0.70      0.64      0.66      6000
weighted avg       0.78      0.80      0.78      6000



2024/09/28 13:15:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
